In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
def capture_sign(image, x, y , w, h, count):
    sign_arr = image[y:y+h,x:x+w,0]
    if sign_arr.shape[0] > 0 and sign_arr.shape[1] > 0:
        cv2.imwrite(f"captures_sign/sign{count}.png", sign_arr)

In [3]:
def detect_sign(imag=[], cascade="stop_cascade.xml", color=(255, 0, 0), thickness=5):
    URL = 'data/traffic_cascades/'
    sign_cascade = cv2.CascadeClassifier(f'{URL}{cascade}')
    imag1 = imag.copy()
    rectangles = sign_cascade.detectMultiScale(imag1)
    count = 0
    for (x,y,w,h) in rectangles:
        capture_sign(imag1,x,y,w,h,count)
        cv2.rectangle(imag1, (x,y), (x+w, y+h), color, thickness)
        count += 1
    return imag1

In [4]:
def run_video_capture():
    capt = cv2.VideoCapture(0)
    while True:
        res,video = capt.read(0)
        if not res:
            print("not res")
            res, video = reset(capt)
        video = detect_sign(video)
        cv2.imshow('Detect traffic signs', video)
        key = cv2.waitKey(1)
        if key == 27:
            break
    capt.release()
    cv2.destroyAllWindows() 

In [5]:
def main():
    run_video_capture()
    
def reset(capture):
    capture.release()
    cv2.destroyAllWindows()
    return cv2.VideoCapture(0).read(0)

In [6]:
main()